In [6]:
import numpy as np
import pandas as pd
import os,sys,glob
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
STUDY_PATH=os.path.join('/NAS','coolio','protocoles','CINOCIS')
SCRIPT_PATH=os.path.join(STUDY_PATH,'scripts')
BIDS_PATH=os.path.join(STUDY_PATH,'BIDS')

In [4]:
participants_file = os.path.join(BIDS_PATH, 'participants.tsv')
participants_df = pd.read_csv(participants_file, sep='\t')
print(participants_df)

    participant_id sex
0          sub-001   F
1          sub-002   F
2          sub-003   F
3          sub-004   F
4          sub-005   M
..             ...  ..
116        sub-129   M
117        sub-130   F
118        sub-131   F
119        sub-132   F
120        sub-133   F

[121 rows x 2 columns]


In [8]:
# Drop duplicate participant_id entries
unique_participants_df = participants_df.drop_duplicates(subset='participant_id')

# Write the unique participants to a new TSV file
unique_participants_file = os.path.join(BIDS_PATH, 'unique_participants.tsv')
unique_participants_df.to_csv(unique_participants_file, sep='\t', index=False)

print(f"Unique participants written to {unique_participants_file}")

Unique participants written to /NAS/coolio/protocoles/CINOCIS/BIDS/unique_participants.tsv


In [5]:
import glob
import csv
from datetime import datetime

# Define the path pattern for the log files
log_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'proc_structural*.txt')

# Define the output CSV file
output_csv_file = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'micapipe_processed_sub.csv')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the pattern
    for log_file in glob.glob(log_files_pattern):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj,f" {ses}", 'proc_structural', ' COMPLETED', ' 09/09', ' romain', ' sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), ' 67.567', ' LOCAL', " v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


In [7]:
# Define the path pattern for the new log files
log_files_pattern_post_structural = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'post_structural*.txt')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the new pattern
    for log_file in glob.glob(log_files_pattern_post_structural):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj, f"{ses}", 'post_structural', 'COMPLETED', '07/07', 'romain', 'sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), '67.567', 'LOCAL', "v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


In [6]:
# Define the path pattern for the new log files
log_files_pattern_surf = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'micapipe_v0.2.0', 'sub-*', 'ses-*', 'logs', 'proc_surf_*.txt')

# Open the CSV file in append mode
with open(output_csv_file, mode='a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Iterate over all log files matching the new pattern
    for log_file in glob.glob(log_files_pattern_surf):
        with open(log_file, 'r') as file:
            content = file.read()
            if "Status          : COMPLETED" in content:
                # Extract SUBJ and SES from the file path
                parts = log_file.split(os.sep)
                subj = parts[-4].split('-')[1]
                ses = parts[-3].split('-')[1]
                
                # Define the row to be added
                row = [subj, f"{ses}", 'proc_surf-fastsurfer', 'COMPLETED', '01/01', 'romain', 'sim', 
                       datetime.now().strftime('%a. %d %b %Y %H:%M:%S CET'), '67.567', 'LOCAL', "v0.2.2 'Northern flicker'"]
                
                # Write the row to the CSV file
                csv_writer.writerow(row)

print(f"Log processing completed and results written to {output_csv_file}")

Log processing completed and results written to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/micapipe_v0.2.0/micapipe_processed_sub.csv


 # Suppression des composantes connexes inférieurs à 4mm

In [10]:
import nibabel as nib
import scipy.ndimage as ndimage

# Define the path pattern for the lesion segmentation files
lesionseg_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3', 'sub-*', 'ses-M*', 'sub-*lesionseg.nii.gz')

# Iterate over all lesion segmentation files matching the pattern
for lesionseg_file in glob.glob(lesionseg_files_pattern):
    # Load the NIfTI file
    img = nib.load(lesionseg_file)
    data = img.get_fdata()
    
    # Get voxel dimensions (in mm)
    voxel_dims = img.header.get_zooms()
    voxel_volume = np.prod(voxel_dims)
    
    # Label connected components
    labeled_array, num_features = ndimage.label(data)
    
    # Count the number of voxels for each connected component
    voxel_counts = ndimage.sum(data, labeled_array, range(1, num_features + 1))
    
    print(f"File: {lesionseg_file}")
    for i, count in enumerate(voxel_counts, start=1):
        volume_mm3 = count * voxel_volume
        print(f"  Component {i}: {count} / {volume_mm3:.2f} mm³")
        # Threshold volume in mm³
        threshold_volume_mm3 = 4.0
        
        # Create a mask for components larger than the threshold
        mask = np.zeros_like(data, dtype=bool)
        for i, count in enumerate(voxel_counts, start=1):
            volume_mm3 = count * voxel_volume
            if volume_mm3 >= threshold_volume_mm3:
                mask |= (labeled_array == i)
        
        # Create a new NIfTI image with the thresholded data
        thresholded_data = data * mask
        thresholded_img = nib.Nifti1Image(thresholded_data, img.affine, img.header)
        
        # Save the new image
        thresholded_file = lesionseg_file.replace('.nii.gz', '_thres4.nii.gz')
        nib.save(thresholded_img, thresholded_file)
        
        print(f"Thresholded image saved to {thresholded_file}")

File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg.nii.gz
  Component 1: 6.0 / 4.71 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
  Component 2: 47.0 / 36.91 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
  Component 3: 148.0 / 116.22 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_thres4.nii.gz
File: /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub-078/ses-M3/sub-078_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_cut_lesionseg.nii.gz
  Component 1: 81.0 / 63.61 mm³
Thresholded image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3/sub

# fusionner les deux volumes (cervical et thoracique coupé)

In [32]:
import nibabel as nib
import numpy as np

# Chemins des fichiers NIfTI à fusionner
file1 = '/NAS/coolio/protocoles/CINOCIS/BIDS/sub-001/ses-M3/anat/sub-001_ses-M3_acq-801T2W_SPAIR2D_T2w.nii.gz'
file2 = '/NAS/coolio/protocoles/CINOCIS/BIDS/sub-001/ses-M3/anat/sub-001_ses-M3_acq-802T2W_SPAIRtho2D_T2w_cut.nii.gz'

# Charger les deux volumes
img1 = nib.load(file1)
img2 = nib.load(file2)

# Extraire les données des volumes
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(f"Dimensions du volume 1 : {data1.shape}")
print(f"Dimensions du volume 2 : {data2.shape}")

# Vérifier que les dimensions des volumes sont compatibles pour la fusion
if data1.shape != data2.shape:
    raise ValueError("Les dimensions des volumes ne correspondent pas et ne peuvent pas être fusionnées.")

# Fusionner les volumes le long de l'axe z (par exemple)
fused_data = np.concatenate((data1, data2), axis=2)

# Créer une nouvelle image NIfTI avec les données fusionnées
fused_img = nib.Nifti1Image(fused_data, img1.affine, img1.header)

# Chemin pour enregistrer le volume fusionné
fused_file = '/NAS/coolio/protocoles/CINOCIS/sub-001_ses-M3_acq-802T2W_SPAIRtho2D_T2w_fused_img2.nii.gz'

# Enregistrer le volume fusionné
nib.save(fused_img, fused_file)

print(f"Volume fusionné enregistré à {fused_file}")

Dimensions du volume 1 : (512, 512, 12)
Dimensions du volume 2 : (12, 512, 451)


ValueError: Les dimensions des volumes ne correspondent pas et ne peuvent pas être fusionnées.

In [28]:
filename = os.path.basename(labeled_file)
print(filename)

AttributeError: module 'os' has no attribute 'filename'

In [33]:
# Define the path pattern for the labeled segmentation files
labeled_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused', 'sub-*', 'ses-M*', 'sub-*_ses-*_acq*_T2w_fus_seg_labeled.nii.gz')

# Iterate over all labeled segmentation files matching the pattern
for labeled_file in glob.glob(labeled_files_pattern):
    # Load the NIfTI file
    img = nib.load(labeled_file)
    data = img.get_fdata()
    
    unique_values = np.unique(data)
    #{len(unique_values)} {unique_values}
    print(f"{os.path.basename(labeled_file)} ")

File: sub-078_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 22 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21.]
File: sub-078_ses-M60_acq-1302T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 22 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21.]
File: sub-126_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 22 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21.]
File: sub-126_ses-M60_acq-302T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 11 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
File: sub-107_ses-M3_acq-1502T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 22 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21.]
File: sub-107_ses-M60_acq-302T2W_SPAIRtho2D_T2w_fus_seg_labeled.nii.gz Number of labels: 22 [ 0.  1.  2.  3.  4.  5.  6.  7.  8. 

### Generation du fichier csv

In [ ]:
import pandas as pd
import glob
import os

# Define the path pattern for the CSV files
csv_files_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'csv', '*.csv')

# Initialize an empty list to store dataframes
dataframes = []

# Iterate over all CSV files matching the pattern
for csv_file in glob.glob(csv_files_pattern):
    # Read the CSV file into a dataframe without headers
    df = pd.read_csv(csv_file, header=None)
    # Transpose the dataframe so that rows become columns
    df_transposed = df.T
    # Set the first row as the header
    df_transposed.columns = df_transposed.iloc[0]
    df_transposed = df_transposed[1:]
    # Add a column with the filename
    df_transposed['filename'] = os.path.basename(csv_file)
    # Append the transposed dataframe to the list
    dataframes.append(df_transposed)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# Drop the specified columns
columns_to_drop = ['adc_mean_lesion_label_0', 'adc_mean_lesion_label_23', 'adc_mean_label_0', 'adc_mean_label_23']
combined_df.drop(columns=columns_to_drop, inplace=True)

# Set the 'filename' column as the index
combined_df.set_index('filename', inplace=True)
print(combined_df)

# Define the output CSV file path
output_csv_path = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'combined_data.csv')

# Save the combined dataframe to a CSV file
combined_df.to_csv(output_csv_path)

print(f"Combined dataframe saved to {output_csv_path}")

0                               vol_label_1  vol_label_2  vol_label_3  \
filename                                                                
sub-086_ses-M3_label_stat.csv    928.161776  1127.614475  1242.260515   
sub-008_ses-M3_label_stat.csv    754.622337  1084.426063   936.014386   
sub-068_ses-M60_label_stat.csv    762.47498  1064.009884  1133.111633   
sub-126_ses-M3_label_stat.csv    802.521975    949.36308  1144.889471   
sub-001_ses-M60_label_stat.csv   910.100822  1041.237006   1165.30597   
...                                     ...          ...          ...   
sub-118_ses-M60_label_stat.csv  1165.306412   1199.07203  2002.379617   
sub-102_ses-M60_label_stat.csv   711.433807   873.194695  1162.950848   
sub-019_ses-M60_label_stat.csv   754.622107  1031.814203    948.57805   
sub-066_ses-M3_label_stat.csv    888.114186  1228.911319  1354.550815   
sub-088_ses-M3_label_stat.csv     810.37481   991.766846    1168.4474   

0                               vol_label_4  vol_l

In [111]:
import nibabel as nib
import numpy as np
import sys
from scipy.ndimage import label
import glob
import os

def write_csv_with_precision(output_csv_path, data_dict, label,precision=4):
    with open(output_csv_path, mode='a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        for label_value, value in data_dict.items():
            writer.writerow([f"{label}_{label_value}", f"{value:.{precision}f}"])

# Usage example:

import csv

STUDY_PATH=os.path.join('/NAS','coolio','protocoles','CINOCIS')
SCRIPT_PATH=os.path.join(STUDY_PATH,'scripts')
BIDS_PATH=os.path.join(STUDY_PATH,'BIDS')

subject =  'sub-001' #sys.argv[1] 
session = 'ses-M60' #sys.argv[2] 

# Chemin vers le fichier CSV de sortie
#output_csv_path = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'csv',f'{subject}_{session}_label_stat.csv')
output_csv_path = os.path.join(BIDS_PATH,f'{subject}_{session}_label_stat.csv')
################
# Volume des labels en mm³
################
# Chemin vers le fichier NIfTI
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused2', subject, session, f'{subject}_{session}_acq-*fus_tho_seg_labeled.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")

label_file = glob.glob(file_pattern)[0]

# Lire le fichier NIfTI
img = nib.load(label_file)
labels_data = img.get_fdata()

# Obtenir les dimensions des voxels (en mm)
voxel_dims = img.header.get_zooms()
voxel_volume = np.prod(voxel_dims)

# Initialiser un dictionnaire pour stocker les volumes des labels en mm³
label_volumes_mm3 = {i: 0 for i in range(1,23)}

# Calculer le volume de chaque label en mm³
for label_value in range(1,23):
    label_volumes_mm3[label_value] = np.sum(labels_data == label_value) * voxel_volume

# Écrire les volumes des labels en mm³ dans un fichier CSV
with open(output_csv_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, volume_mm3 in label_volumes_mm3.items():
        writer.writerow([f"vol_label_{label_value}", volume_mm3])

#write_csv_with_precision(output_csv_path, label_volumes_mm3,"vol_label_",precision=0)
# write_csv_with_precision(output_csv_path, fa_mean_per_lesionlabel)
# write_csv_with_precision(output_csv_path, adc_mean_per_label)
# write_csv_with_precision(output_csv_path, adc_mean_per_lesionlabel)

################
# Nombre et volume de lesion par label
################
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused2', subject, session, f'{subject}_{session}_acq-*fus_tho_lesionseg.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")
lesion_file = glob.glob(file_pattern)[0]
# Lire le fichier NIfTI
img = nib.load(lesion_file)
lesion_data = img.get_fdata()

# Initialiser un dictionnaire pour stocker le nombre et le volume des composantes connexes
lesion_stats = {i: {'count': 0, 'volume_voxels': 0, 'volume_mm3': 0} for i in range(1,23)}

# Calculer le nombre et le volume de chaque composante connexe pour chaque label
for label_value in range(1,23):
    binary_data = lesion_data[np.where(labels_data == label_value)]
    print(binary_data.shape)
    labeled_array, num_features = label(binary_data)
    print(f"Label {label_value}: {num_features} features")
    lesion_stats[label_value]['count'] = num_features
    lesion_stats[label_value]['volume_voxels'] = np.sum(binary_data)
    lesion_stats[label_value]['volume_mm3'] = lesion_stats[label_value]['volume_voxels'] * voxel_volume

# Écrire les statistiques des lésions dans le fichier CSV
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in lesion_stats.items():
        writer.writerow([f"lesion_count_label{label_value}", stats['count']])
        
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in lesion_stats.items():
        writer.writerow([f"lesion_vol_label{label_value}",stats['volume_mm3']])

################
# FA moyenne par label
################
# Chemin vers le fichier NIfTI
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused2', subject, session, f'{subject}_{session}_acq-*fus_tho_seg_labeled_4fa.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")
fa_file = glob.glob(file_pattern)[0]
img = nib.load(fa_file)
labels4fa_data = img.get_fdata()

file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'DWI', subject, session, f'{subject}_{session}_acq-*_dwi_fa.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")
fa_file = glob.glob(file_pattern)[0]
img = nib.load(fa_file)
fa_data = img.get_fdata()

# Initialiser un dictionnaire pour stocker les fa moyenne par label
fa_mean_per_label = {i: 0 for i in range(1,23)}

for label_value in range(1,23):
    fa_mean_per_label[label_value] = np.mean(fa_data[(labels4fa_data == label_value)])

    # Écrire les statistiques des lésions dans le fichier CSV
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in fa_mean_per_label.items():
        writer.writerow([f"fa_mean_label_{label_value}", stats])

################
# FA moyenne des volumes lesionnelles par label
################
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused2', subject, session, f'{subject}_{session}_acq-*fus_tho_lesionseg_4fa.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")
lesion4fa_file = glob.glob(file_pattern)[0]
img = nib.load(lesion4fa_file)
lesion4fa_data = img.get_fdata()

# Initialiser un dictionnaire pour stocker les fa moyenne par label
fa_mean_per_lesionlabel = {i: 0 for i in range(1,23)}

for label_value in range(1,23):
    fa_mean_per_lesionlabel[label_value] = np.mean(fa_data[(labels4fa_data == label_value) & (lesion4fa_data == 1)])

    # Écrire les statistiques des lésions dans le fichier CSV
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in fa_mean_per_lesionlabel.items():
        writer.writerow([f"fa_mean_lesion_label_{label_value}", stats])


#########
#pareil pour le MD
#############
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'DWI', subject, session, f'{subject}_{session}_acq-*_dwi_adc.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")
adc_file = glob.glob(file_pattern)[0]
img = nib.load(adc_file)
adc_data = img.get_fdata()

# Initialiser un dictionnaire pour stocker les fa moyenne par label
adc_mean_per_label = {i: 0 for i in range(24)}

for label_value in range(1,23):
    adc_mean_per_label[label_value] = np.mean(adc_data[np.where(labels4fa_data == label_value)])

    # Écrire les statistiques des lésions dans le fichier CSV
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in adc_mean_per_label.items():
        writer.writerow([f"adc_mean_label_{label_value}", stats])

# Initialiser un dictionnaire pour stocker les fa moyenne par label
adc_mean_per_lesionlabel = {i: 0 for i in range(24)}

for label_value in range(1,23):
    adc_mean_per_lesionlabel[label_value] = np.mean(adc_data[(labels4fa_data == label_value) & (lesion4fa_data == 1)])

    # Écrire les statistiques des lésions dans le fichier CSV
with open(output_csv_path, mode='a', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for label_value, stats in adc_mean_per_lesionlabel.items():
        writer.writerow([f"adc_mean_lesion_label_{label_value}", stats])


print(f"Les statistiques ont été écrites dans le fichier {output_csv_path}")


/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_seg_labeled.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg.nii.gz
(1159,)
Label 1: 24 features
(1326,)
Label 2: 18 features
(1484,)
Label 3: 24 features
(1845,)
Label 4: 20 features
(1669,)
Label 5: 0 features
(1448,)
Label 6: 0 features
(1360,)
Label 7: 0 features
(1400,)
Label 8: 0 features
(1358,)
Label 9: 0 features
(1092,)
Label 10: 0 features
(1156,)
Label 11: 0 features
(1258,)
Label 12: 0 features
(1117,)
Label 13: 0 features
(1232,)
Label 14: 0 features
(1149,)
Label 15: 0 features
(1153,)
Label 16: 0 features
(1338,)
Label 17: 0 features
(2197,)
Label 18: 0 features
(2643,)
Label 19: 0 features
(926,)
Label 20: 0 features
(0,)
Label 21: 0 features
(0,)
Label 22: 0 features
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/

/home/global/Anaconda3-2023.09/envs/conda_env_3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/global/Anaconda3-2023.09/envs/conda_env_3.12/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# Chemin vers le fichier NIfTI
file_pattern = os.path.join(BIDS_PATH, 'derivatives_v2.0', 'sct_4.3_fused2', subject, session, f'{subject}_{session}_acq-*fus_tho_seg_labeled_std.nii.gz')
print(f"{glob.glob(file_pattern)[0]}")

label_file = glob.glob(file_pattern)[0]

# Lire le fichier NIfTI
img = nib.load(label_file)
labels_data = img.get_fdata()


for label_value in range(1,23):
    binary_data = lesion_data[np.where(labels_data == label_value)]
    print(lesion_data.shape)
    labeled_array, num_features = label(binary_data)
    
    print(labeled_array.shape)
    # Create a new NIfTI image with the labeled data
    labeled_img = nib.Nifti1Image(labeled_array, img.affine, img.header)
    
    # Define the output file path
    labeled_file = lesion_file.replace('.nii.gz', f'_label_{label_value}.nii.gz')
    
    # Save the new image
    nib.save(labeled_img, labeled_file)
    print(labeled_file)
    print(f"Labeled image saved to {labeled_file}")
    print(f"Label {label_value}: {num_features} features")

(12, 512, 963)
(1159,)
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg_label_1.nii.gz
Labeled image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg_label_1.nii.gz
Label 1: 24 features
(12, 512, 963)
(1326,)
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg_label_2.nii.gz
Labeled image saved to /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg_label_2.nii.gz
Label 2: 18 features
(12, 512, 963)
(1484,)
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-001/ses-M60/sub-001_ses-M60_acq-1502T2W_SPAIRtho2D_T2w_fus_tho_lesionseg_label_3.nii.gz
Labeled image saved to /NAS/coolio/protocol